In [1]:
from tensorflow.keras.models import load_model
model = load_model('Input your model')

In [ ]:
from IPython.display import Audio 
from IPython.core.display import display
def sound():
    display(Audio('alarm-no3.mp3', autoplay=True))
# sound()

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

url = "IP Webcam address/video"
cap = cv2.VideoCapture(url)

fps = cap.get(cv2.CAP_PROP_FPS)  # fps = 25.0

for f in glob('capture/*.jpg'):
    os.remove(f)

L = []  # For counting impassible road
while(True):
    camera, frame = cap.read()

    # # frame resize
    # resize_frame = cv2.resize(frame, (250, 250), interpolation=cv2.INTER_AREA)

    # frame crop & resize
    resize_frame = cv2.resize(frame[:, 420:1500], (250, 250), interpolation=cv2.INTER_AREA)

    if resize_frame is not None:
        
        # Capture 5 images per 1 second
        if (int(cap.get(1)) % 5 == 0):
            # Save image as 001 ~ 005.jpg
            filename = f'{str(len(os.listdir("capture"))%5 + 1).zfill(3)}.jpg'
            cv2.imwrite(f'capture/{filename}', resize_frame)
            
        if len(os.listdir('capture')) >= 5:
            
            df = pd.DataFrame({'image_path': glob('capture/*.jpg')})
            
            test_datagen = ImageDataGenerator(rescale=1/255.)
            test_generator = test_datagen.flow_from_dataframe(
                df,
                x_col='image_path',
                y_col='None',
                target_size=(250, 250),
                class_mode=None,
                shuffle=False
            )    
            
            pred = model.predict_generator(test_generator)
            pred = np.argmax(pred, axis=-1)

            if np.all(pred == 1):
                cv2.putText(resize_frame, f"Passible", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            elif np.all(pred == 0):
                cv2.putText(resize_frame, "Imassible", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                L.append(np.all(pred == 0))  # Append 1 if all of predictions 0
                
            for f in glob('capture/*.jpg'):
                os.remove(f)
                
        cv2.imshow("Frame", resize_frame)

    # Detect impassible road more than 3 times -> sound()
    if L.count(1) > 2:
        sound()
        break
    
    # To finish program(close windows), press q                     
    q = cv2.waitKey(1)
    if q == ord("q"):
        break
cv2.destroyAllWindows()